# Import library

In [ ]:
import logging
import sys

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# Xóa các handler có sẵn
if logger.hasHandlers():
    logger.handlers.clear()

# Handler cho console
console = logging.StreamHandler(sys.stdout)
console.setLevel(logging.DEBUG)

# Handler để ghi vào file (nếu cần)
fileh = logging.FileHandler("kaggle.log", mode="w")
fileh.setLevel(logging.INFO)

# Tạo formatter rõ ràng
fmt = logging.Formatter(
    "%(asctime)s %(levelname)s:%(name)s:%(lineno)d - %(message)s",
    "%Y-%m-%d %H:%M:%S"
)
console.setFormatter(fmt)
fileh.setFormatter(fmt)

# Gắn handler
logger.addHandler(console)
logger.addHandler(fileh)

logger.debug("✅ Logger đã sẵn sàng trên Kaggle notebook")

In [ ]:
pip install -U transformers underthesea

In [ ]:
import pickle 
import torch
import numpy as np
import pandas as pd
import re
# import nltk
# nltk.download('punkt_tab')

from underthesea import word_tokenize

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from datasets import load_dataset, Dataset, DatasetDict

from tqdm.auto import tqdm

from matplotlib import pyplot as plt

In [ ]:
import transformers
print(transformers.__version__)

In [ ]:
!pip list

In [ ]:
!python

In [ ]:
import sys
for module in sys.modules:
    try:
        print(module,sys.modules[module].__version__)
    except:
        try:
            if  type(modules[module].version) is str:
                print(module,sys.modules[module].version)
            else:
                print(module,sys.modules[module].version())
        except:
            try:
                print(module,sys.modules[module].VERSION)
            except:
                pass

# Loading data

In [ ]:
#Load from csv
# print(phmtrngiaph_vietnamesetextsummaries_path)
df = pd.read_csv('../input/vietnamese-text-summaries/data_cleaned.csv', encoding = "utf-8")
df = df[df["ratio"] <= 0.5 ]
# df = df[df["sentences"] <= 6 ]

# contents = df["input"]
# summaries = df["output"]

In [ ]:
# #Load from pkl
# # with open ('neg.pkl', 'rb') as fp:
# #     contents = pickle.load(fp)

# #Load from csv
# df = pd.read_csv(f'{haianhjobs_textsummarizationvietnamese1_path}/Dataset_articles_NoID-6.csv', encoding = "utf-8")
# df = df.dropna()
# df = df[["Summary", "Contents"]]

# contents = df["Contents"]
# summaries = df["Summary"]

# Data Exploration

In [ ]:
df.describe().T
# df.count()
# df.info()

In [ ]:
import seaborn as sns
columns = ["sentences", "input_sentences", "ratio"]

plt.figure(figsize=(20, 5))
for i in range(3):
    plt.subplot(1, 3, i+1)
    sns.histplot(df[columns[i]], bins=50)
    plt.xlabel(columns[i])
    plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(20, 5))
for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.boxplot(df[columns[i]], vert=False)
    plt.grid(True)
    plt.xlabel(columns[i])
    # plt.ylabel('')
plt.show()

In [ ]:
df[df["input"].astype(str) == '5 trường hợp được rút bảo hiểm xã hội một lầnĐiều kiện rút bảo hiểm xã hội một lần là chủ đề được người lao động rất quan tâm trong năm 2023, khi các ban ngành, địa phương tiến hành lấy ý kiến góp ý cho dự án Luật Bảo hiểm xã hội (sửa đổi).Tuy nhiên, dự kiến Luật Bảo hiểm xã hội (sửa đổi) sẽ thông qua vào giữa năm 2024 và đến ngày 1/7/2025 mới có hiệu lực thi hành. Do đó, trong năm 2024, điều kiện để người lao động hưởng bảo hiểm xã hội một lần vẫn được áp dụng theo Luật Bảo hiểm xã hội năm 2014 và Nghị quyết 93/2015/QH13.Theo Khoản 1 Điều 60 Luật Bảo hiểm xã hội 2014, người lao động chỉ được rút bảo hiểm xã hội một lần trong 4 trường hợp. Nghị quyết 93/2015/QH13 bổ sung thêm một trường hợp là người lao động sau 1 năm ngừng tham gia bảo hiểm xã hội và chưa đủ 20 năm đóng bảo hiểm xã hội.Mức hưởng bảo hiểm xã hội một lần năm 2024Theo Khoản 2 Điều 60 Luật Bảo hiểm xã hội 2014, mức hưởng bảo hiểm xã hội một lần được tính theo số năm đã đóng bảo hiểm xã hội theo 2 giai đoạn.Với những năm đóng bảo hiểm xã hội trước năm 2014, cứ mỗi năm được tính bằng 1,5 tháng mức bình quân tiền lương tháng đóng bảo hiểm xã hội.Với những năm đóng bảo hiểm xã hội từ năm 2014 trở đi, cứ mỗi năm được tính bằng 2 tháng mức bình quân tiền lương tháng đóng bảo hiểm xã hội.Trường hợp thời gian đóng bảo hiểm xã hội chưa đủ một năm thì mức hưởng bảo hiểm xã hội bằng số tiền đã đóng, mức tối đa bằng 2 tháng mức bình quân tiền lương tháng đóng bảo hiểm xã hội.Nếu chỉ tính đơn thuần về mức đóng - hưởng, người lao động lựa chọn rút bảo hiểm xã hội một lần thiệt thòi rất nhiều.Hiện nay, mỗi tháng, người lao động đóng bảo hiểm xã hội vào quỹ hưu trí, tử tuất bằng 8% mức lương cơ bản, người sử dụng lao động đóng 14%, tổng cộng là 22%; mỗi năm là 2,64 tháng lương cơ bản.Như vậy, khi rút bảo hiểm xã hội một lần, người lao động thiệt 1,14 tháng lương cho mỗi năm đóng bảo hiểm xã hội trước năm 2014 và thiệt 0,64 tháng lương cho mỗi năm đóng bảo hiểm xã hội từ năm 2014 trở đi.Theo BHXH Việt Nam, trong thực tế đã có rất nhiều người sau khi nhận bảo hiểm xã hội một lần muốn nộp lại tiền để phục hồi số năm đã tham gia bảo hiểm xã hội cho đủ điều kiện hưởng lương hưu. Tuy nhiên, hiện pháp luật về bảo hiểm xã hội chưa quy định trường hợp này.Có nghĩa là, nếu đã hưởng bảo hiểm xã hội một lần thì quá trình đóng trước đó không còn. Khi đi làm trở lại, người lao động phải tham gia bảo hiểm xã hội lại từ đầu. Vì vậy, người lao động nên cân nhắc kỹ trước khi quyết định rút bảo hiểm xã hội một lần.']

In [ ]:
pd.set_option('display.max_colwidth', None)
df.head(30)
# print(max(df["sentences"]))
# df[df['input_sentences']==5]

# Model

In [ ]:
tokenizer= AutoTokenizer.from_pretrained("vinai/bartpho-word")
model= AutoModelForSeq2SeqLM.from_pretrained("vinai/bartpho-word")

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

model_path = "VietAI/ViT5-base"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Pre-processing

Dữ liệu CSV -> DataFrame -> Dataset -> Split train test

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
valid_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)

train_contents = train_df["input"].tolist()
train_summaries = train_df["output"].tolist()

valid_contents = _df

test_contents = test_df["input"].tolist()
test_summaries = test_df["output"].tolist()

print(train_contents[5])
print(train_summaries[5])

In [ ]:
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2, seed=42)
val_test_dataset = dataset["test"].train_test_split(test_size=0.5, seed=42)

dataset = DatasetDict(
    {
        "train": dataset["train"], 
        "valid": val_test_dataset["train"], 
        "test": val_test_dataset["test"]
    }
)
# type(dataset["train"]["input"])
dataset

In [ ]:
train_contents = dataset["train"]["input"]
train_summaries = dataset["train"]["output"]

valid_contents = dataset["valid"]["input"]
valid_summaries = dataset["valid"]["output"]

test_contents = dataset["test"]["input"]
test_summaries = dataset["test"]["output"]

test_summaries[0]

## BARTpho

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

text = 'Ngày 2/4, ông Dũng và bà Nguyễn Thị Hồng Nga, 42 tuổi, nguyên phó chủ tịch xã Bảo Quang, bị Công an Đồng Nai khởi tố, cho tại ngoại, về hành vi Thiếu trách nhiệm gây hậu quả nghiêm trọng. Nguyễn Minh Phúc, 46 tuổi, nguyên công chức Tư pháp - Hộ tịch, UBND xã Bảo Quang, bị bắt tạm giam về tội Lạm quyền trong khi thi hành công vụ. Đào Thị Liễu (42 tuổi, vợ của ông Dũng) đã giả chữ ký, dấu vân tay của chủ đất để ký hợp đồng chuyển nhượng cho người khác, rồi nhờ Nguyễn Minh Phúc tiếp nhận chứng thực, chiếm đoạt tài sản của nhiều người với số tiền hơn 11 tỷ đồng. Ngày 9/6/2023, Liễu bị Công an TP Long Khánh bắt tạm giam về tội Lừa đảo chiếm đoạt tài sản.'
# tokens = word_tokenize(text, format="text")
tokens = tokenizer(text, max_length=256, truncation=True, padding="max_length")
print(tokens)

In [ ]:
def preprocess_bart(dataset):
    contents = ["summarize:" + word_tokenize(content, format="text") + "</s>" for content in dataset["input"]]
    summaries = [word_tokenize(summary, format="text") for summary in dataset["output"]]
    contents_tokenized = tokenizer(contents, max_length=512, truncation=True, padding="max_length")
    summaries_tokenized = tokenizer(summaries, max_length=256, truncation=True, padding="max_length")
    dataset_tokenized = {
        "input_ids": contents_tokenized["input_ids"],
        "attention_mask": contents_tokenized["attention_mask"],
        "labels": summaries_tokenized["input_ids"],
    }
    return dataset_tokenized

# Specify the original columns to remove from the individual datasets
columns_to_remove = ["input", "output", "sentences", "input_sentences", "ratio", "__index_level_0__"]

dataset = dataset.map(preprocess_bart, remove_columns=columns_to_remove, batched=True)
dataset

## ViT5

In [ ]:
def preprocess_t5(dataset):
    contents = ["summarize:" + content + "</s>" for content in dataset["input"]]
    summaries = dataset["output"]
    contents_tokenized = tokenizer(contents, max_length=512, truncation=True, padding="max_length")
    summaries_tokenized = tokenizer(summaries, max_length=256, truncation=True, padding="max_length")
    dataset_tokenized = {
        "input_ids": contents_tokenized["input_ids"],
        "attention_mask": contents_tokenized["attention_mask"],
        "labels": summaries_tokenized["input_ids"],
    }
    return dataset_tokenized

# Specify the original columns to remove from the individual datasets
columns_to_remove = ["input", "output", "sentences", "input_sentences", "ratio", "__index_level_0__"]

dataset = dataset.map(preprocess_t5, remove_columns=columns_to_remove, batched=True)
dataset

# Training

In [ ]:
model.device

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!pip install evaluate rouge_score
import evaluate
rouge=evaluate.load('rouge')

def compute_metrics(eval_preds):
    """Định nghĩa độ đo"""
    predicts, labels = eval_preds

    predicts = tokenizer.batch_decode(predicts, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    scores = rouge.compute(predictions=predicts, references=labels, use_stemmer=True)

    # return {key: value for key, value in scores.items()}
    return scores

## BARTpho

In [ ]:
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

os.environ["WANDB_DISABLED"] = "true"

training_args = Seq2SeqTrainingArguments(
    output_dir="checkpoints",
    logging_dir="logs",
    eval_strategy="steps",
    save_strategy="steps",
    logging_strategy="steps",

    eval_steps=0.1,
    save_steps=0.1,
    logging_steps=0.1,
    
    learning_rate=1e-5, 
    # warmup_ratio=0.1,
    per_device_train_batch_size=2,  # Reduced batch size
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4, # Added gradient accumulation
    weight_decay=0.01,
    # optim="adamw_torch",

    save_total_limit=3,
    num_train_epochs=3,

    load_best_model_at_end=True, # 🔥 Enables saving the best checkpoint
    metric_for_best_model="eval_loss", # Metric to track (e.g., "eval_loss", "eval_rougeL")
    greater_is_better=False, # Set to True if higher metric is better (e.g., Rouge), False if lower (e.g., loss)
    
    # predict_with_generate=True,
    # generation_max_length=1024,
    # generation_num_beams=4,
    
    fp16=True,
    report_to=None
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["valid"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
    # compute_metrics=compute_metrics
)

trainer.train()

## ViT5

In [ ]:
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

os.environ["WANDB_DISABLED"] = "true"

training_args = Seq2SeqTrainingArguments(
    output_dir="checkpoints",
    logging_dir="logs",
    eval_strategy="steps",
    save_strategy="steps",
    logging_strategy="steps",

    eval_steps=0.1,
    save_steps=0.1,
    logging_steps=0.1,
    
    learning_rate=2e-5,
    # warmup_ratio=0.1,
    per_device_train_batch_size=4,  # Reduced batch size
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4, # Added gradient accumulation
    weight_decay=0.01,
    # optim="adamw_torch",

    save_total_limit=3,
    num_train_epochs=3,

    load_best_model_at_end=True, # 🔥 Enables saving the best checkpoint
    metric_for_best_model="eval_loss", # Metric to track (e.g., "eval_loss", "eval_rougeL")
    greater_is_better=False, # Set to True if higher metric is better (e.g., Rouge), False if lower (e.g., loss)
    
    # predict_with_generate=True,
    # generation_max_length=1024,
    # generation_num_beams=4,
    
    fp16=True,
    report_to=None
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["valid"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
    # compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
pd.DataFrame(trainer.state.log_history)

In [ ]:
logs_df = pd.DataFrame(trainer.state.log_history)
training_loss = logs_df["loss"].dropna().tolist()
valid_loss = logs_df["eval_loss"].dropna().tolist()
# valid_loss
logs_df = logs_df[["epoch","step","loss"]].dropna()
logs_df["eval_loss"] = valid_loss
logs_df

In [ ]:
training_loss = [1.1500,
                    1.1152,
                    1.0307,
                    0.9480,
                    0.9076,
                    0.9485,
                    0.8953,
                    0.8629,
                    0.8760]

valid_loss = [1.025203,
                0.966157,
                0.923553,
                0.921797,
                0.914168,
                0.904691,
                0.903724,
                0.899221,
                0.894148]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

# Giả lập dữ liệu: 10 epoch
epochs = np.arange(1, 10)

# Tạo figure và axis
fig, ax = plt.subplots(figsize=(10, 5))

# Vẽ đồ thị
ax.plot(epochs, training_loss, label='Training Loss')
ax.plot(epochs, valid_loss, label='Validation Loss')

# Formatter giúp scale nhãn x sang 0→3
formatter = FuncFormatter(lambda val, pos: f"{(val - 1) / 10 * 3:.2f}")
ax.xaxis.set_major_formatter(formatter)

# Đặt tick tại 5 điểm đều trên dữ liệu gốc
ticks = np.linspace(1, 10, 10)
ax.set_xticks(ticks)

# Thiết lập nhãn và tiêu đề
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('BARTpho')

# Hiển thị legend và lưới
ax.legend()
ax.grid(True)

# Hiển thị đồ thị
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# from matplotlib.ticker import FuncFormatter

plt.figure(figsize=(10, 5))

plt.plot([training_loss[i] for i in range(len(training_loss))], label=f'Training loss')
plt.plot([valid_loss[i] for i in range(len(valid_loss))], label=f'Validation loss')
plt.xticks(range(0, 10), np.round(np.linspace(0.3, 3.0, 10), 2))
# plt.xlim(-1, 10)

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('ViT5')

plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Define the directory to save the model
# output_dir = "/kaggle/working/bartphomodel_1"
output_dir = "/kaggle/working/vit5model_1"
# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Save the model and tokenizer
trainer.save_model(output_dir)

print(f"Model saved to {output_dir}")

# To load the model later
# loaded_tokenizer = AutoTokenizer.from_pretrained(output_dir)
# loaded_model = AutoModelForSeq2SeqLM.from_pretrained(output_dir)
# loaded_model.to(device) # Make sure to move the loaded model to the correct device

In [ ]:
!zip -r bartphomodel_1.zip bartphomodel_1

In [ ]:
!zip -r vit5model_11.zip vit5model_1

In [ ]:
%cd /kaggle/working
from IPython.display import FileLink
FileLink(r'vit5model_11.zip')

# Generate summaries

In [ ]:
# saved_model_path = "../input/vit5model/transformers/default/1/vit5model_4_1"
# saved_model_path = "../input/bartphomodel/transformers/default/1/bartphomodel_1"
saved_model_path = "../input/vit5model/transformers/default/2/vit5model_1"
# saved_model_path = "../input/vit5model-4/vit5model_4"
# saved_model_path = "vit5model_1"

tokenizer = AutoTokenizer.from_pretrained(saved_model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(saved_model_path)

model

In [ ]:
model_path = "VietAI/ViT5-base"
tokenizer_base = AutoTokenizer.from_pretrained(model_path)
model_base = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [ ]:
tokenizer_basesum = AutoTokenizer.from_pretrained("VietAI/vit5-base-vietnews-summarization")  
model_basesum = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base-vietnews-summarization")

In [ ]:
def generate_summary(text, tokenizer, model):
    model.to("cuda")
    
    input_text = "summarize: " + text + "</s>"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)
    output_ids = model.generate(input_ids, max_length=1024, num_beams=4, early_stopping=True)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

model.to("cuda")
batch_size = 4

def summarize(contents, tokenizer, model):
    summaries = []
    
    for i in tqdm(range(0, len(contents), batch_size), desc="Generating"):
        batch_contents = contents[i:i+batch_size]
        for j in range (len(batch_contents)):
            batch_contents[j] = "summarize: " + batch_contents[j] + "</s>"
            
        contents_tokenize = tokenizer(batch_contents, return_tensors = "pt", padding=True)
        input_ids = contents_tokenize["input_ids"].to(model.device)
        # print(len(input_ids[0]))
        attention_mask = contents_tokenize["attention_mask"].to(model.device)

        with torch.no_grad():
            output_ids = model.generate(
                input_ids=input_ids, 
                attention_mask=attention_mask,
                num_beams=4,
                max_length=1024,
                early_stopping=True
            )
        # print(len(output_ids[0]))
        
        # for output in output_ids:
        #     sentence = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        #     print("\n",sentence)
        decoded = tokenizer.batch_decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    summaries.extend(decoded)
    return summaries
    # return re.sub(r"[^a-zA-ZÀ-ỹ0-9\s\.,;:!?()\"'-]", "", summary)


# generated_summaries = []
# for i in range (len(test_contents)):
#     summary = summarize(test_contents[i])
#     generated_summaries.append(summary)

# print("Generate summaries complete.")


def summarize_text_t5(text, tokenizer, model):
    # Prepare input text with the "summarize:" prefix
    input_text = "summarize: " + text
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)

    # Generate summary
    summary_ids = model.generate(input_ids, max_length=1024, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decode and return the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return summary

In [ ]:
model.device

In [ ]:
def summarize_t5(contents, tokenizer, model):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    summaries = []
    
    for i in tqdm(range(len(contents)), desc="Generating"):
        content = "summarize: " + contents[i] + "</s>"
        content_tokenize = tokenizer(content, return_tensors = "pt")
        
        input_ids = content_tokenize["input_ids"].to(model.device)
        # print(f"len {i}:",len(input_ids[0]))
        attention_mask = content_tokenize["attention_mask"].to(model.device)

        with torch.no_grad():
            output_ids = model.generate(
                input_ids=input_ids, 
                attention_mask=attention_mask,
                num_beams=4,
                max_length=1024,
                # length_penalty=0.3,
                no_repeat_ngram_size=20,
                early_stopping=True,
            )
        # print(len(output_ids[0]))
        
        # for output in output_ids:
        #     sentence = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        #     print("\n",sentence)
        summary = tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        summaries.append(summary)
        
    return summaries

In [ ]:
 def summarize_bart(contents, tokenizer, model):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    summaries = []
    
    for i in tqdm(range(len(contents)), desc="Generating"):
        content = "summarize: " + word_tokenize(contents[i], format="text") + "</s>"
        content_tokenize = tokenizer(content, truncation=True, max_length=1024,
                                     return_tensors = "pt")
        
        input_ids = content_tokenize["input_ids"].to(model.device)
        # print(f"len {i}:",len(input_ids[0]))
        attention_mask = content_tokenize["attention_mask"].to(model.device)

        with torch.no_grad():
            output_ids = model.generate(
                input_ids=input_ids, 
                attention_mask=attention_mask,
                num_beams=4,
                max_length=1024,
                # length_penalty=0.3,
                no_repeat_ngram_size=10,

                early_stopping=True,
            )
        # print(len(output_ids[0]))
        
        summary = tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        summary = summary.replace('_', ' ')
        summary = re.sub(r'"\s*([^"]*?)\s*"', r'"\1"', summary)


        summaries.append(summary)
        
    return summaries

In [ ]:
generated_summaries

In [ ]:
text = ['Sẽ giảm số năm đóng bảo hiểm xã hội tối thiểuTrước kỳ họp thứ 6 Quốc hội khóa XV, cử tri tỉnh Bắc Kạn và tỉnh Vĩnh Long đã có các kiến nghị về việc thời gian tham gia bảo hiểm xã hội tự nguyện 20 năm như hiện tại là quá lâu, dẫn đến việc người dân không có khả năng đóng phí.Các cử tri kiến nghị  Bộ Lao động - Thương binh và Xã hội cần nghiên cứu, tham mưu Chính phủ trình Quốc hội xem xét sửa đổi Luật Bảo hiểm xã hội, giảm thời gian tham gia đóng bảo hiểm xã hội tự nguyện.Đồng thời, đề nghị quy định giảm số năm tối thiểu đóng bảo hiểm xã hội xuống 15 năm, 10 năm để được hưởng lương hưu, mức lương hưu tương ứng với thời gian đóng bảo hiểm xã hội.Người dân giải quyết thủ tục bảo hiểm xã hội (Ảnh minh họa: BHXH VN).Quy định các chế độ hưởng lương hưu phù hợp theo nguyên tắc đóng nhiều hưởng nhiều, đóng ít hưởng ít, để tạo cơ hội cho những người tham gia muộn hoặc quá trình tham gia không liên tục có thời gian đóng bảo hiểm xã hội ngắn được hưởng lương hưu.Từ đó khuyến khích người tham gia, hạn chế được tình trạng nhận bảo hiểm xã hội một lần do chưa đủ số năm đóng bảo hiểm xã hội bắt buộc.Trả lời nội dung này, Bộ Lao động - Thương binh và Xã hội cho hay, Ban Chấp hành Trung ương Đảng đã có nghị quyết số 28 năm 2018 về cải cách chính sách bảo hiểm xã hội.Trong đó có nội dung, sửa đổi điều kiện hưởng chế độ hưu trí theo hướng giảm dần số năm đóng bảo hiểm xã hội tối thiểu để được hưởng chế độ hưu trí từ 20 năm xuống 15 năm, hướng tới còn 10 năm.Với mức hưởng được tính toán phù hợp nhằm tạo điều kiện cho người lao động cao tuổi, có số năm tham gia bảo hiểm xã hội thấp được tiếp cận và thụ hưởng quyền lợi bảo hiểm xã hội.Hiện nay, Chính phủ đã trình Quốc hội dự án Luật Bảo hiểm xã hội sửa đổi. Dự thảo luật sửa đổi đã đề xuất giảm điều kiện về số năm đóng bảo hiểm xã hội tối thiểu để được hưởng lương hưu từ 20 năm xuống 15 năm.Trường hợp người lao động khi đến tuổi hưu mà chưa đủ điều kiện hưởng lương hưu và cũng chưa đủ tuổi hưởng trợ cấp hưu trí xã hội thì được lựa chọn hưởng trợ cấp hằng tháng do quỹ bảo hiểm xã hội chi trả cho thời gian trước khi đủ tuổi hưởng trợ cấp hưu trí xã hội tùy thuộc vào thời gian đóng, tiền lương, thu nhập đóng của người lao động.Đồng thời, trong thời gian hưởng trợ cấp hằng tháng thì được hưởng bảo hiểm y tế do ngân sách nhà nước đảm bảo.Như vậy, nội dung kiến nghị của cử tri sẽ tiếp tục được trao đổi, thảo luận trong quá trình các cơ quan nghiên cứu, tiếp thu ý kiến của các vị đại biểu Quốc hội để hoàn thiện dự thảo luật sửa đổi, báo cáo Quốc hội khóa XV xem xét, thông qua tại kỳ họp thứ 7 (tháng 6/2024).Sửa quy định tiền lương đóng bảo hiểm xã hộiCử tri tỉnh Bắc Giang cũng nêu kiến nghị như sau: thực tế nhiều doanh nghiệp chi trả tiền công cho người lao động cao nhưng số tiền làm căn cứ đóng bảo hiểm xã hội lại rất thấp, chỉ bằng mức lương tối thiểu vùng.Dự thảo Luật Bảo hiểm xã hội bổ sung quy định tiền lương làm căn cứ đóng bảo hiểm xã hội bắt buộc (Ảnh minh họa: Mạnh Quân).Trả lời nội dung này, Bộ Lao động - Thương binh và Xã hội nêu rõ, Chính phủ đã trình Quốc hội dự án Luật Bảo hiểm xã hội sửa đổi.Trong đó, để khắc phục tình trạng chậm đóng, trốn đóng bảo hiểm xã hội bắt buộc, dự thảo luật sửa đổi đã bổ sung quy định trách nhiệm của các cơ quan trong việc xác định và quản lý đối tượng thuộc diện tham gia bảo hiểm xã hội.Cùng với đó, sửa đổi, bổ sung nhiều biện pháp xử lý, chế tài xử lý tình trạng chậm đóng, trốn đóng bảo hiểm xã hội.Dự luật cũng bổ sung quy định tiền lương làm căn cứ đóng bảo hiểm xã hội bắt buộc thấp nhất, quy định cụ thể hơn theo hướng tiền lương làm căn cứ đóng bảo hiểm xã hội là tiền lương tháng, bao gồm mức lương, phụ cấp lương, các khoản bổ sung khác được trả thường xuyên và ổn định trong mỗi kỳ trả lương.',
       'Ngày 2/4, ông Dũng; bà Nguyễn Thị Hồng Nga, 42 tuổi, nguyên phó chủ tịch xã Bảo Quang, bị Công an Đồng Nai khởi tố, cho tại ngoại, về hành vi Thiếu trách nhiệm gây hậu quả nghiêm trọng. Liên quan vụ án, Nguyễn Minh Phúc, 46 tuổi, nguyên công chức Tư pháp - Hộ tịch, UBND xã Bảo Quang, bị bắt tạm giam về tội Lạm quyền trong khi thi hành công vụ.Theo điều tra, từ năm 2021, Đào Thị Liễu (42 tuổi, vợ của ông Dũng) nhận giấy chứng nhận quyền sử dụng đất của một số hộ làm dịch vụ tách thửa, chuyển đổi mục đích sử dụng đất... Sau đó, Liễu giả chữ ký, dấu vân tay của chủ đất ký hợp đồng chuyển nhượng cho người khác rồi mang đến UBND xã Bảo Quang nhờ Nguyễn Minh Phúc tiếp nhận chứng thực.Cơ quan điều tra cáo buộc, Phúc là công chức Tư pháp – Hộ tịch được phân công tiếp nhận chứng thực sao y và chứng thực chữ ký, không được giao nhiệm vụ tiếp nhận hồ sơ chứng thực hợp đồng, nhưng biết Liễu là vợ của ông Dũng nên người này đã tiếp nhận hồ sơ rồi trình ông Dũng, bà Nga ký chứng thực. Bằng thủ đoạn trên, Liễu đã chiếm đoạt tài sản của nhiều người với số tiền hơn 11 tỷ đồng.Ngày 9/6/2023, Liễu bị Công an TP Long Khánh bắt tạm giam về tội Lừa đảo chiếm đoạt tài sản. Vụ án sau đó được chuyển cho Công an tỉnh Đồng Nai thụ lý điều tra.']
# print("ViT5 base:\n",summarize(text, tokenizer_base, model_base))
# print("\nViT5 finetuned 2:\n") 
# generated_summaries = summarize_bart(text, tokenizer, model)
generated_summaries = summarize_t5(text, tokenizer, model)

# generated_summaries = summarize_t5(test_contents, tokenizer, model)
# generated_summaries = summarize_bart(test_contents, tokenizer, model)
print("Generate summaries successfully!")
# print("\nViT5 base summarization:\n",summarize(text, tokenizer_basesum, model_basesum))

In [ ]:
generated_summaries

In [ ]:
i = 1
# # # print("Content:\n",text[i])
# print("Content:\n",test_contents[i])
# print("\nReference:\n",test_summaries[i])
generated_summaries1[i] = re.sub(r'\s+([/%;:])', r'\1', generated_summaries1[i])
generated_summaries1[i] = re.sub(r'\(\s*([^)]*?)\s*\)', r'(\1)', generated_summaries1[i])

print("\nBARTpho:\n",generated_summaries1[i])
print("\nViT5:\n",generated_summaries2[i])


In [ ]:
with open('bartpho_summaries_word.pkl', 'wb') as f:
    pickle.dump(generated_summaries, f)

print("Pickle file created.")

In [ ]:
print(len(test_contents))

In [ ]:
n = 0.75
# print("Content:", contents.iloc[0]) # Use iloc to access by position (df type)
# print("Train contents:",train_contents[0])
print("Test contents:",test_df.iloc[0]["input"])
print("Test summary:",test_df.iloc[0]["output"])

# print(f"\n Summary {n} sentences: ", generated_summaries[0])
# print("\nOriginal summary: ", test_summaries[0])

# print(textrank_summaries.keys())

# ROUGE Scores

In [ ]:
# prompt: store dictionaries for next time without re running

# Save the dictionaries to files
# with open('gemeraeted_summaries.pkl', 'wb') as f:
#     pickle.dump(kmeans_summaries, f)

# To load the dictionaries later:
with open('../input/generated-summaries/bartpho_summaries_word.pkl', 'rb') as f:
    generated_summaries1 = pickle.load(f)
with open('../input/generated-summaries/vit5_summaries_base.pkl', 'rb') as f:
    generated_summaries2 = pickle.load(f)

# print("Pickle file loaded. List has", len(generated_summaries), "summaries.")
# for i in range(5):
#     print(f"\nGenerated summaries {i}: ",generated_summaries[i])

In [ ]:
!pip install rouge-score
from rouge_score import rouge_scorer, scoring


scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def rouge(reference_sums, generated_sums):
#   rouge1_scores, rouge2_scores, rougeL_scores = [], [], []
    rouge_scores = []

    for reference_sum, generated_sum in zip(reference_sums, generated_sums):
        scores = scorer.score(reference_sum, generated_sum)
        rouge_scores.append([scores['rouge1'].fmeasure, scores['rouge2'].fmeasure, scores['rougeL'].fmeasure])

    return np.mean(rouge_scores, axis=0)

def batch_rouge(reference_sums, generated_sums, batch_size):
    rouge_scores = []
    for i in tqdm(range(0, len(reference_sums), batch_size), desc="Calculating ROUGE"):
        batch_reference_sums = reference_sums[i:i+batch_size]
        batch_generated_sums = generated_sums[i:i+batch_size]

        batch_rouge_scores = rouge(batch_reference_sums, batch_generated_sums)
        rouge_scores.append(batch_rouge_scores)

    return np.mean(rouge_scores, axis=0)

def batch_rouge_parallel(reference_sums, generated_sums, batch_size):
    rouge_scores = Parallel(n_jobs=-1)(delayed(rouge)(reference_sums[i:i + batch_size], generated_sums[i:i + batch_size]) for i in range(0, len(reference_sums), batch_size))
    return np.mean(rouge_scores, axis=0)

In [ ]:
rouge = evaluate.load('rouge')

def rouge_score(reference_sums, generated_sums):
    rouge_score = []
    scores = rouge.compute(predictions=generated_sums, references=reference_sums, use_stemmer=True)
    rouge_score.append([scores['rouge1'], scores['rouge2'], scores['rougeL']])

    return rouge_score

In [ ]:
rouge_scores_summaries = []
rouge_scores_summaries = batch_rouge([train_summaries[5]], generated_summaries, 1)

In [ ]:
rouge_scores_summaries = []
rouge_scores_summaries = batch_rouge(test_summaries, generated_summaries2, 100)
rouge_scores_summaries

In [ ]:
rouge_scores_summaries = []
rouge_scores_summaries = batch_rouge([test_summaries[i]], [generated_summaries1[i]], 100)
rouge_scores_summaries

In [ ]:
rouge_scores_2 = rouge_score(test_summaries, generated_summaries)

rouge_scores_2

In [ ]:
df_rouge = pd.DataFrame([rouge_scores_summaries], columns = ["ROUGE-1", "ROUGE-2", "ROUGE-L"])
# df_rouge.loc[1] = rouge_scores_summaries
# df_rouge.loc[2] = rouge_scores_2[0]
df_rouge

In [ ]:
rouge_scores_kmeans = {}
rouge_scores_textrank = {}
rouge_scores_centroid = {}

for n in np.linspace(0.15, 0.75, 7):
    n = round(n, 2)
    rouge_scores_kmeans[n] = batch_rouge_parallel(summaries, kmeans_summaries[n], 100)
    rouge_scores_textrank[n] = batch_rouge_parallel(summaries, textrank_summaries[n], 100)
    rouge_scores_centroid[n] = batch_rouge_parallel(summaries, centroid_summaries[n], 100)

    # rouge_scores_kmeans[n] = rouge_score(summaries.reset_index(drop=True), kmeans_summaries[n])
    # rouge_scores_textrank[n] = rouge_score(summaries.reset_index(drop=True), textrank_summaries[n])
    # rouge_scores_centroid[n] = rouge_score(summaries.reset_index(drop=True), centroid_summaries[n])

In [ ]:
with open('rouge_scores_kmeans.pkl', 'rb') as f:
    rouge_scores_kmeans = pickle.load(f)

with open('rouge_scores_textrank.pkl', 'rb') as f:
    rouge_scores_textrank = pickle.load(f)

with open('rouge_scores_centroid.pkl', 'rb') as f:
    rouge_scores_centroid = pickle.load(f)

In [ ]:
# with open('rouge_scores_kmeans.pkl', 'wb') as f:
#     pickle.load(rouge_scores_kmeans, f)

# with open('rouge_scores_textrank.pkl', 'wb') as f:
#     pickle.dump(rouge_scores_textrank, f)

# with open('rouge_scores_centroid.pkl', 'wb') as f:
#     pickle.dump(rouge_scores_centroid, f)

In [ ]:
for n in range(2,7):
    print(f"\n---\ROUGE Scores for {n} sentences:")

    print(f"{'':<20} {'ROUGE-1':<10} {'ROUGE-2':<10} {'ROUGE-L':<10}")
    print(f"{'K-Means':<20} {rouge_scores_kmeans[n][0]:<10.4f} {rouge_scores_kmeans[n][1]:<10.4f} {rouge_scores_kmeans[n][2]:<10.4f}")
    # print("\n")
    print(f"{'TextRank':<20} {rouge_scores_textrank[n][0]:<10.4f} {rouge_scores_textrank[n][1]:<10.4f} {rouge_scores_textrank[n][2]:<10.4f}")
    print(f"{'Centroid-based':<20} {rouge_scores_centroid[n][0]:<10.4f} {rouge_scores_centroid[n][1]:<10.4f} {rouge_scores_centroid[n][2]:<10.4f}")

In [ ]:
# df_rouge = pd.DataFrame(columns=['n sentences',('KMeans','1','2'), 'TextRank'])

# Define MultiIndex for columns
columns = pd.MultiIndex.from_tuples([
    ('KMeans', 'ROUGE-1'), ('KMeans', 'ROUGE-2'), ('KMeans', 'ROUGE-L'),
     ('TextRank', 'ROUGE-1'), ('TextRank', 'ROUGE-2'), ('TextRank', 'ROUGE-L'),
     ('Centroid-based', 'ROUGE-1'), ('Centroid-based', 'ROUGE-2'), ('Centroid-based', 'ROUGE-L'),
])

df_rouge = pd.DataFrame(index = [n for n in range(2,7)],columns=columns)

for n in range(2,7):
    df_rouge.loc[n] = list(rouge_scores_kmeans[n]) + list(rouge_scores_textrank[n]) + list(rouge_scores_centroid[n])

df_rouge.style.set_table_styles([
    {'selector': 'th', 'props': [('border', '1px solid black')]},  # Add border around the header
    {'selector': 'td', 'props': [('border', '1px solid black')]},  # Add border around the cells
])
# df_rouge

In [ ]:
columns = pd.MultiIndex.from_tuples([
    ('ROUGE-1', 'KMeans'), ('ROUGE-1', 'TextRank'), ('ROUGE-1', 'Centroid-based'),
     ('ROUGE-2', 'KMeans'), ('ROUGE-2', 'TextRank'), ('ROUGE-2', 'Centroid-based'),
     ('ROUGE-L', 'KMeans'), ('ROUGE-L', 'TextRank'), ('ROUGE-L', 'Centroid-based'),
])

df_rouge_2 = pd.DataFrame(columns=columns)

for n in np.linspace(0.15, 0.75, 7):
    n = round(n, 2)
    df_rouge_2.loc[n,'ROUGE-1'] = rouge_scores_kmeans[n][0], rouge_scores_textrank[n][0], rouge_scores_centroid[n][0]
    df_rouge_2.loc[n,'ROUGE-2'] = rouge_scores_kmeans[n][1], rouge_scores_textrank[n][1], rouge_scores_centroid[n][1]
    df_rouge_2.loc[n,'ROUGE-L'] = rouge_scores_kmeans[n][2], rouge_scores_textrank[n][2], rouge_scores_centroid[n][2]

# df_rouge_2 = df_rouge_2.T

df_rouge_2.style.set_table_styles([
    {'selector': 'th', 'props': [('border', '1px solid black')]},  # Add border around the header
    {'selector': 'td', 'props': [('border', '1px solid black')]},  # Add border around the cells
])

In [ ]:
from matplotlib import pyplot as plt


for i in range(3):
    plt.figure(figsize=(10, 5))

    plt.plot([rouge_scores_kmeans[round(n,2)][i] for n in np.linspace(0.15, 0.75, 7)], label=f'K-Means')
    plt.plot([rouge_scores_textrank[round(n,2)][i] for n in np.linspace(0.15, 0.75, 7)], label=f'TextRank')
    plt.plot([rouge_scores_centroid[round(n,2)][i] for n in np.linspace(0.15, 0.75, 7)], label=f'Centroid-based')
    # plt.ylim(0.24, 0.71)
    plt.xticks(range(0,7),[round(n,2) for n in np.linspace(0.15, 0.75, 7)])
    plt.xlabel('Generated / Original')
    plt.ylabel('F1-Score')
    plt.title(f'ROUGE-{i+1}')
    if i==2: plt.title('ROUGE-L')
    plt.legend()
    plt.grid(True)
plt.show()